# Test Imperfect Lorenz Model 

In [2]:
from pathlib import Path
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

# Add src folder to Python path
sys.path.append(str(Path("../src").resolve()))

from data_generation import generate_data
from lstm import WindowedTimeSeries, train_model, predict_one_step, predict_autoregressive
from ode_model import LorenzModel

In [3]:
data = generate_data(10000, noise_level=0.05)

In [4]:
model = LorenzModel()

In [5]:
preds = model.predict(y0=data, t_step=0.05)

In [6]:
time_step = 0.05
c_range = np.linspace(12,16,9)
k_range = [1,5,10,15,20]

for k in k_range:
  true = data
  true = true[k: , :]
  for c in c_range:
    model.update_params(c=c)
    preds = model.predict(y0=data, t_step=time_step*k)

    preds = preds[:-k, :]
    mse = np.mean((preds - true)**2)
    print(f"k:{k}, c:{c:.2f}, MSE:{mse}")



k:1, c:12.00, MSE:1.6586726260420226
k:1, c:12.50, MSE:1.5639914038306657
k:1, c:13.00, MSE:1.4879894467968589
k:1, c:13.50, MSE:1.426425172780258
k:1, c:14.00, MSE:1.37613213506818
k:1, c:14.50, MSE:1.33472196150607
k:1, c:15.00, MSE:1.30037502663936
k:1, c:15.50, MSE:1.2716916420459345
k:1, c:16.00, MSE:1.2475852628181303
k:5, c:12.00, MSE:11.216726890885631
k:5, c:12.50, MSE:8.011286234380142
k:5, c:13.00, MSE:6.24607702600688
k:5, c:13.50, MSE:5.153403598919918
k:5, c:14.00, MSE:4.419062760232013
k:5, c:14.50, MSE:3.898875478186054
k:5, c:15.00, MSE:3.5164387804466637
k:5, c:15.50, MSE:3.2272717999487495
k:5, c:16.00, MSE:3.0037190859170275
k:10, c:12.00, MSE:39.3157543072601
k:10, c:12.50, MSE:26.754144246430325
k:10, c:13.00, MSE:23.16848924513056
k:10, c:13.50, MSE:21.63624667388116
k:10, c:14.00, MSE:20.698450049052568
k:10, c:14.50, MSE:20.071990012135778
k:10, c:15.00, MSE:19.630981748621547
k:10, c:15.50, MSE:19.309089521515393
k:10, c:16.00, MSE:19.06774522540572
k:15, c:12

C = 13 matches MSE of LSTM for prediction of step = 10

In [7]:
time_step = 0.05


for c in range(10,21):
  model.update_params(c=c)
  preds = model.predict_scipy(y0=data, t_step=time_step*10)
  true = data

  preds = preds[:-10, :]
  true = true[10: , :]
  mse = np.mean((preds - true) ** 2)
  print(f"c: {c}, MSE: {mse}")

c: 10, MSE: 373.0202885364678
c: 11, MSE: 144.44360185662836
c: 12, MSE: 38.066647839741705
c: 13, MSE: 22.890700093440955
c: 14, MSE: 20.49220926843374
c: 15, MSE: 19.45088052337387
c: 16, MSE: 18.89895576038702
c: 17, MSE: 18.575182207045803
c: 18, MSE: 18.37283996225855
c: 19, MSE: 18.241035123187473
c: 20, MSE: 18.152612656070342
